# Disruption Analysis Compilation

#### File preparation for ArcGIS Analysis

Here we will prepare original source Vehicle Location History files for disruption analysis in ArcGIS with the workflow outlined below:

- import and slice CSV file(s) to result in dataframe with only relevant fields to analysis.
- Construct dataframe datetime fields
- Construct disruption index functions for generating a disruption index field in the dataframe.
- Export the dataframe to a new CSV file for spatial analysis in GIS.

The processes will be outlined in more detail in their own sections.

In [1]:
# Importing relevant libraries
import pandas as pd
import os
import glob
import numpy as np
import warnings
from datetime import datetime

## Import Data

File data will be imported from CSVs provided by Michael Long at Capital Metropolitan Transportation Authority. Initial data will not be provided and subsequent data will be stripped of identifiers for bus and driver identification. The only relevant data for our analysis lies in the headway time of vehicles, and time and location of record. 

In [2]:
# Setting up path vars.
path = r'../00_Source_Data/Capital_Metro/Vehicle_Location_History' # Relative source file path
all_files = glob.glob(os.path.join(path , "*.csv")) # all files

In [3]:
# This block generates a list of dataframes where each item in the list is one file.
li = []

with warnings.catch_warnings():
    warnings.simplefilter('ignore') # ignore dataframe generation warnings
    for filename in all_files:
        df = pd.read_csv(filename, index_col=None, header=0)
        li.append(df)

## Construct the Dataframe

We concatenate the list of dataframes from all files in the `Vehicle_Location_History` folder into a single dataframe and slice that dataframe to contain just the information we need for analysis.

In [4]:
# This block generates a dataframe with all data from the files stored in the source file path.

frame = pd.concat(li, axis=0, ignore_index=True)

#### Formatting the Dataframe

The new dataframe we're interested in only needs the following fields:
- **timecentral**: Datetime at CST formatted as YYYYMMHHmmssss
- **lat**: Latitude the reading was taken at in WKID 4326 or WGS 1984
- **lon**: Longitude the reading was taken at in WKID 4326 or WGS 1984
- **headwaysecs**: Headway reading in seconds
- **scheduledheadwaysecs**: Planned headway in seconds

Fields that will be added later will be explained in the Disruption Index section.

In [5]:
# Saving just the fields we're interested in and removing the original data from memory.
data = frame[['timecentral', 'lat', 'lon', 'headwaysecs', 'scheduledheadwaysecs']]
del frame, li, path, all_files, df

#### Converting to Datetime

Our current time field is formatted as an integer. In order to use the time values we will want to convert them into a datetime object. The current format is `20220625185013`: \
`2022` - `%Y` Year with century as a decimal \
`06`   - `%m` Month as a zero-padded decimal \
`25`   - `%d` Day of the month as a zero-padded decimal \
`18`   - `%H` Hour (24-hour clock) as a zero-padded decimal \
`50`   - `%M` Minute as a zero-padded decimal \
`13`   - `%S` Second as a zero-padded decimal

So the format to change into datetime is `%Y%m%d%H%M%S`

We can aggregate the records of the data in the requested time intervals, taking the mean location (`lat`, `lon`), mean headway values (`headwaysecs`) and the mean `disruption index` later.

In [6]:
# Lets convert the integer times into datetime objects

data['timecentral'] = pd.to_datetime(data['timecentral'], format='%Y%m%d%H%M%S')

data['timecentral'].head()

0   2022-06-19 00:00:23
1   2022-06-19 00:00:03
2   2022-06-19 00:00:16
3   2022-06-19 00:00:48
4   2022-06-19 00:00:43
Name: timecentral, dtype: datetime64[ns]

In [30]:
# Lets remove rows with NaN values for `headwaysecs` as they will not be useful for our analysis

data = data[data['headwaysecs'].notna()]
data = data[data['scheduledheadwaysecs'].notna()]

In [31]:
print(data[['headwaysecs']].max()[0],',', data[['headwaysecs']].min()[0])

7199.8 , 0.0


In [32]:
print(data[['scheduledheadwaysecs']].max()[0],',', data[['scheduledheadwaysecs']].min()[0])

7680.0 , 34.0


In [38]:
# Check to make sure it worked
print(data['headwaysecs'].isna().value_counts())
print(data['scheduledheadwaysecs'].isna().value_counts())

False    4129586
Name: headwaysecs, dtype: int64
False    4129586
Name: scheduledheadwaysecs, dtype: int64


## Construct Disruption Index

In this section, we will create new fields for the disruption indices and calculate them based on the work of Federico Malucelli and Emanuele Tresoldi in their case study. DOI: [s12469-019-00196-y](https://link.springer.com/article/10.1007/s12469-019-00196-y)

The fields we will construct are: \
`HR`     - Headway Ratio: the ratio between the observed headway and the planned one. \
`HS`     - Headway Standard Deviation: standard deviation of the difference between the observed and the planned headways \
`PR`     - Percentage Regularity Deviation: the percentage average ratio between the deviation of the observed headway from the planned one and the planned headway. \
`PW`     - Piece-wise linear regularity index




First, we will define a frequency with which to do our sampling. For a list of the appropriate strings to enter into the frequency variable please see [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases).

In [ ]:
# Define a frequency to sample the data with.
freq='H'

In [34]:
# Before we an do any calculations that require time intervals, we need to calculate the gap of every record.
# For this, we'll create a new field `gap` that will simply be the difference between the `headwaysecs` and
# the `scheduledheadwaysecs`.

data['gap'] = data['headwaysecs'] - data['scheduledheadwaysecs']

# We also need a field of values that will give us the ratio of observed to expected headways. We'll call this
# field `headwayratio`.

data['headwayratio'] = data['headwaysecs']/data['scheduledheadwaysecs']

In [39]:
# Create the left side of the dataframe grouped in the chosen interval. 
# This will be concatenated with the calculated fields later and serve as the master dataframe that we'll export later.

master_df = data.groupby(pd.Grouper(key='timecentral', freq=freq)).mean()
master_df

,lat,lon,headwaysecs,scheduledheadwaysecs,gap,headwayratio
timecentral,,,,,,
2022-06-19 00:00:00,30.293191,-97.737928,1687.359947,1693.142857,-5.782910,1.006356
2022-06-19 01:00:00,30.295484,-97.738471,1686.653455,1715.270608,-28.617152,0.980554
2022-06-19 02:00:00,30.278059,-97.737004,1732.625760,1692.547033,40.078726,1.020753
2022-06-19 03:00:00,30.279623,-97.735773,1741.818209,1710.132159,31.686050,1.019602
2022-06-19 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2022-06-25 19:00:00,30.291210,-97.732976,1465.227817,1405.872931,59.354886,1.085319
2022-06-25 20:00:00,30.292363,-97.732183,1667.768094,1610.051929,57.716165,1.050490
2022-06-25 21:00:00,30.292783,-97.734055,1752.594771,1702.955922,49.638849,1.035930


#### Headway Statistic Functions

Lets start by making the function that will generate the three statistical fields for our dataframe. These functions are as follows:
$$ HR=1-\bigg | \dfrac{\sum_{q \in P} \dfrac{v_o (q)}{v_e (q)}}{|P|} -1 \bigg | = 1-\bigg | \dfrac{\sum_{\text{interval}} \text{ratio}}{\text{records in interval}} -1 \bigg |\\
HS=1- \dfrac{\text{std({$v_o (q), \forall q \in P$})}}{\text{avg({$v_e (q), \forall q \in P$})}} \\
PR=1- \dfrac{\sum_{q \in P} \dfrac{|v_o (q) - v_e (q)|}{v_e (q)}}{|P|} = 1- \dfrac{\sum_{\text{interval}} \dfrac{|\text{gap}|}{v_e (q)}}{\text{records in interval}}$$

In [ ]:
# Here we define a function to calculate the headway ratio over a given interval of time.
## Default sample interval is hourly. Interval should be in strftime format.

def headway_stats(df, observed, expected, interval="H"):
    
    

First, lets consider the gap $x(q)$ in the planned $(v_e (q))$ and observed $(v_o (q))$ headways:
$$ x(q) = v_o (q) - v_e (q) $$

A negative value is an early pass and a positive value is a delay.

Malucelli and Tresoldi then define the function of $f(x(q))$ of the gap as:
$$
f(x(q))=
\begin{cases}
-\alpha x(q) & \quad \text{if $x(q) < -\theta_1$}\\
0 & \quad \text{if $-\theta_1 \leq x(q) < \theta_2$}\\
\beta x(q) & \quad \text{if $\theta_2 \leq x(q) < \theta_3$}\\
\gamma x(q)+\delta & \quad \text{if $\theta_3 \leq x(q)$}\\
\end{cases}
$$

Where $\theta_1$, $\theta_2$, $\theta_3$ and $\alpha, \beta, \gamma, \delta$ are suitable parameters. The function is 0 if the pass is regular and is not 0 if the pass is irregular. We can ignore contributions of earliness on the index by setting $\alpha = 0$. Likewise, if we set all coefficients to 0 and $\delta = 1$ we are left with the simple index the Azienda Trasporti Milanesi used at the writing of their paper. Values where $x(q) \geq \theta_3$ are intended to penalize large gaps more than the equivlent sum of small gaps.

Thus the index of regularity based on Malucelli and Tresoldi's piece-wise function is:
$$
I(PW)= \sum_{q \in P} f(x(q))
$$

Where $P$ is the set of all passes in a given period.

In consideration of time, we will only be constructing the piece-wise function from Malucelli and Tresoldi and using its related index for the purposes of analysis.

In [ ]:
# Here we define a function to calculate the gap of a given record.
## The Default values are set to the simple index with a 2 minute threshhold of irregularity.

def PW_Index(observed, expected, alpha=0, beta=0, gamma=0, delta=1, late=-180, early=180):
    if observed - expected